In [1]:
import requests
import csv
from ltr.data import CorpusApi, Config

In [2]:
queries = CorpusApi.getValidationQueriesAsDict()

In [3]:
evaluationMap = {}

with open('data/validation/docv2_trec2020_qrels.txt', 'r') as txt:
    txtReader = csv.reader(txt, delimiter=' ')
    for line in txtReader:
        queryMap = evaluationMap.get(line[0], {})
        forwardIndexList = queryMap.get(line[3],[])
        forwardIndexList.append(line[2])
        queryMap[line[3]] = forwardIndexList
        reverseIndexMap = queryMap.get('reverse', {})
        reverseIndexMap[line[2]] = line[3]
        queryMap['reverse'] = reverseIndexMap
        
        evaluationMap[line[0]] = queryMap
        

In [73]:
from typing import List


def reciprocalRank(queryId: int, evalList: List) -> float:
    
    rank = 1
    
    lookupMap = evaluationMap[queryId].get('reverse')
    
    for docId in evalList:
        v = lookupMap.get(docId)
        if v is None or int(v) == 0:
            rank += 1
        else:
            break

    return 1/rank if rank <= 100 else 0
    

In [78]:
def averagePrecision(queryId: int, evalList: List) -> float:
    
    lookupMap = evaluationMap[queryId].get('reverse')
    
    foundDocs = 0
    
    sum = 0
    
    for i, docId in enumerate(evalList):
        v = lookupMap.get(docId)
        if v is not None and int(v) != 0:
            foundDocs += 1
            sum += (foundDocs / (i + 1))
            
    
    return sum / len(evalList)

In [80]:
for key in list(evaluationMap.keys())[:]:
    try:
        # get query
        query = queries[key]
        
        # get result from solr
        request = {
            "fields": "id",
            "limit": 100,
            "params": {
                "qf": "title headings body",
                "defType": "dismax",
                "q": query
            }
        }
        
        response = requests.post(f'http://localhost:8983/solr/thesis-ltr/select', json=request)
        
        queryResult = [doc['id'] for doc in response.json()["response"]["docs"]]
        
        
        # calculate metrics
        # Reciprocal Rank
        rr = reciprocalRank(key, queryResult)
        evaluationMap[key]['rr'] = rr
        print(f'RR: {rr}')
        # Normalized Discounted Cumulative Gains (NDCG)
        # Normalized Cumulative Gains (NCG)
        # Average Precision (AP)
        ap = averagePrecision(key, queryResult)
        evaluationMap[key]['ap'] = ap
        print(f'AP: {ap}')
        
        print(f'processed {key} query: {query}')
    except KeyError as err:
        # ignore the queries that are not in the validation queries list
        pass

RR: 0
AP: 0.0
processed 42255 query: average salary for dental hygienist in nebraska
RR: 0.3333333333333333
AP: 0.022007575843370254
processed 47210 query: average wedding dress alteration cost
RR: 0.3333333333333333
AP: 0.003952020202020202
processed 67316 query: can fever cause miscarriage early pregnancy
RR: 0.2
AP: 0.010152380952380951
processed 135802 query: definition of laudable
RR: 0.125
AP: 0.0025357142857142857
processed 156498 query: do google docs auto save
RR: 1.0
AP: 0.04267696640945941
processed 169208 query: does mississippi have an income tax
RR: 0.5
AP: 0.0067267080745341615
processed 174463 query: dog day afternoon meaning
RR: 0.05555555555555555
AP: 0.00971629457852368
processed 258062 query: how long does it take to remove wisdom tooth
RR: 0.3333333333333333
AP: 0.005399510041840813
processed 330975 query: how much would it cost to install my own wind turbine
RR: 1.0
AP: 0.0703447719340717
processed 332593 query: how often to button quail lay eggs
RR: 0
AP: 0.0
pro